In [1]:
# Importing Necessary dependencies
import pandas as pd
import snowflake.connector as sf
import numpy as np
import time
import boto3
import os
import zipfile
import smtplib, ssl
import shutil
import csv
from snowflake.connector.pandas_tools import write_pandas

In [2]:
# Create folder locally for analysis
def FolderCreation():
    try:
        # Create Local Directory to store files in temporarily
        os.mkdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')
        os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')
        root_directory = os.getcwd()
        claimslog.append('Successfully created CTL File Temporary Folder at '
                         + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    except Exception as e:
        claimslog.append('There was an error with creating the temporary CTL File - ' + str(e))

        

# Create Connection Object for Connecting to AWS
def AWSConnection():
    try:
        s3 = boto3.resource(
            service_name='s3',
            region_name='us-east-1',
            aws_access_key_id=os.getenv('aws_access_key_id'),
            aws_secret_access_key=os.getenv('aws_access_key_id'))
        claimslog.append('AWS connection object created at '
                         + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        return s3
    except Exception as e:
        claimslog.append('There was an error with creating the AWS connection object - ' + str(e)) 


# Search S3 Bucket and Download Files Locally        
def S3SearchAndDownload(connection,filename_list):
    #Creating a list to store all the keys (file names) to download
    key_list = []

    try:
        #Searching the S3 bucket for the most current Ping Files
        for obj in connection.Bucket('hometeam-clinical-data').objects.all():
            for filename_format in filename_format_list:
                if filename_format in str(obj):
                    #print(obj.key)
                    key_list.append(obj.key)

    except Exception as e:
        claimslog.append('There was an error while looking for most CCA Files - ' + str(e))
        
    try:
        #Downloading each of the files found in the key list
        for file in key_list:
            s3.Bucket('hometeam-clinical-data').download_file(file,file.split('/')[2])
        claimslog.append('Files were successfully downloaded at ' 
                         + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    except Exception as e:
        claimslog.append('There was an error while trying to download the CTL Files - ' + str(e))
        
        
# Unzipping files 
def UnzipFiles():
    # On local computer, change directory and set directory for unzipping of files.
    os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')
    root_directory = os.getcwd()
    
    # Locate only Zipped Files
    files_to_unzip = []
    for filename in os.listdir(root_directory):
        if 'zip' in filename:
            files_to_unzip.append(filename)
    try:
        # Unzip each file in the Zipped files list
        for zipped_file in files_to_unzip:
            with zipfile.ZipFile(root_directory+"\\"+zipped_file, 'r') as zip_ref:
                # print(zipped_file)
                zip_ref.extractall(root_directory)
        shutil.unpack_archive(root_directory+"\\"+zipped_file,root_directory+"\\"+zipped_file.split('.')[0])
        claimslog.append('Successfully Unzipped each file at ' 
                         + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    except Exception as e:
        claimslog.append('There was an error while trying to unzip each file - ' + str(e))
        
# CTL Files Dictionary
def CreateDFDict():
    # Create dictionary to store dataframes as they are created 
    df_dict = {}
    error_dict= {}

    # Set current directory
    cwd = os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')

    # Loop through all txt files in the directory
    for i,file in enumerate(os.listdir(cwd)):
        if '.txt' in file:

            # Empty lists to story the data while cleaning
            df_list = []
            df_error_list = []

            # Open the txt file
            with open(file, 'r') as csvfile:
                reader = csv.reader(csvfile,delimiter='|')

                # Read through each line and find any rows with errors
                for i,row in enumerate(reader):
                    if i == 0:
                        # Capture the correct number of columns for the dataframe
                        correct_columns = len(row)

                    df_list.append(row)

                    # Create list of rows with errors
                    if len(row) < correct_columns:
                        df_error_list.append(i)

                # Review error list
                if df_error_list:

                    # Check to see if the list row in the data frame is an empty row, if so, drop it
                    if len(df_list[df_error_list[-1]]) == 0:
                        df_error_list.pop()

                    # The error exists between two rows, so looking at the second occurance of an error 
                    # and deleting the first item should fix the error
                    for i,error in enumerate(df_error_list):
                        if i % 2 != 0:
                            df_list[error].pop(0)

                    # Loop back through the error list and join first errors to second errors to make a complete row
                    for i,error in enumerate(df_error_list):
                        if i % 2 == 0:
                            df_list[error] = df_list[error] + df_list[error + 1]

                # Loop back through the entire data frame list to delete those rows that are smaller than the correct
                # number of columns
                for i,item in enumerate(df_list):
                    if len(item) < correct_columns:
                        del df_list[i]        

                df = pd.DataFrame(df_list[1:])
                df.columns = df_list[0]
                df = df.astype(str)
                df.columns = map(lambda x: str(x).upper(), df.columns)
                df_dict[csvfile.name.replace(' ','_').split(".")[0]] = df
                error_dict[csvfile.name.replace(' ','_').split(".")[0]] = df_error_list

            csvfile.close()

    for key,value in df_dict.items():
        print(key)

    print(error_dict)

    return df_dict

# Delete all files in local folder
def FolderDeletion():
    os.chdir('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')
    shutil.rmtree('C:\\Users\\Jad Driggers\\Documents\\Vesta\\CTLFILES')
    claimslog.append('Successfully Deleted all contents in temporary CTL Folder at ' + 
                     time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    
# Create a dictionary that stores the columns in the data frame and the max length of the values in those columns
# in order to create table in snowflake to minimize table size
def SQLTableStructure():
    #Create two dictionaries to store the columns and the max len of values in those columns
    max_col_len = {}
    col_dict = {}

    #Vectorizing the lenth function
    measurer = np.vectorize(len)

    #Looping through df_dictionary to capture column names and max len of values in those columns
    max_col_len = {}
    for key,value in df_dict.items():
        col_len = measurer(df_dict[key].astype(str)).max(axis=0)
        max_col_len[key] = col_len
        col_dict[key]= df_dict[key].columns.tolist()

    #Function for joining the two dictionaries with similar keys (claim files)    
    def common_entries(*dcts):
        if not dcts:
            return
        for i in set(dcts[0]).intersection(*dcts[1:]):
            yield (i,) + tuple(d[i] for d in dcts)
    mylist = list(common_entries(col_dict,max_col_len))

    #Creating new dictionary and zipping the column names with respective max len of values in those columns
    sql_dict = {}
    for x in mylist:
        sql_dict[x[0]] = list(zip(x[1],x[2]))

    #Iterating through the list values to prep for SQL to Snowflake
    sql_script_dict_table = {}
    for key,value in sql_dict.items():
        script_string_table = ''
        for (col,max_len) in sql_dict[key]:
            script_string_table += str(col) + ' VARCHAR(' + str(max_len+10) + '),'
        sql_script_dict_table[key] = "(" + script_string_table[:-1] + ")" 

    return sql_script_dict_table    

# Create Snowflake Object
def SnowflakeConnection():
    #Creating of parameters for securing connection to Snowflake
    username = os.getenv('Snowflake_User')
    password = os.getenv('Snowflake_password')
    account = os.getenv('Snowflake_account')'

    #Define parameters if neccessary
    warehouse = 'DEVELOPER_STANDARD'
    database = 'VESTA_DEVELOPMENT'
    schema = 'ANALYST_SANDBOX'

    #Create connection object for Snowflake connection
    conn = sf.connect(user=username, password=password, account=account, warehouse=warehouse)
    return conn

# Snowflake execution function
def execute_query(connection,query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close

In [3]:
#Create Log for review
start_time = time.time()
claimslog = []

# Create local folder
FolderCreation()

# Create AWS connection object
s3 = AWSConnection()

# Create the filename format for locating the proper CTL files to parse
filename_format_list = ['CL_VESTA_Claims_Monthly_202203081839.txt',
                        'CL_Vesta_Pharmacy_Catchup_202203081328.txt',
                        'CL_VESTA_Enrollment_202203081316.txt']

# Search AWS for files and download locally
S3SearchAndDownload(s3,filename_format_list)

# Not neccessary with CTL
# UnzipFiles()

# Create dataframe dictionary of CTL files
df_dict = CreateDFDict()

# Delete all local CTL files and local CTL folder
FolderDeletion()

# Detemine the table structure and size for Snowflake
sql_table = SQLTableStructure()

# # Creating of parameters for securing connection to Snowflake
# username = 'mwhitener'
# password = 'Vesta@0104'
# account = 'vh83029.us-east-1'

# # Define parameters if neccessary
# warehouse = 'DEVELOPER_STANDARD'
# database = 'VESTA_DEVELOPMENT'
# schema = 'ANALYST_SANDBOX'

# # Create connection object for Snowflake connection
# conn = sf.connect(user=username, password=password, account=account, warehouse=warehouse)

# # Define Database to use in Snowflake
# sql = 'USE DATABASE {}'.format(database)
# execute_query(conn,sql)
    
# # Define Schema to use in Snowflake
# sql = 'USE SCHEMA {}.{}'.format(database,schema)
# execute_query(conn,sql)
    
# # Define Warehouse to use in Snowflake
# sql = 'USE WAREHOUSE {}'.format(warehouse)
# execute_query(conn,sql)

# # Create CTL_ENROLL_RAW
# try:
#     sql = 'CREATE TABLE IF NOT EXISTS CTL_ENROLL_RAW_TEST ' + sql_table['CL_VESTA_Enrollment_202203081316']
#     execute_query(conn,sql)
#     success, nchucks, nrows, _ = write_pandas(conn,df_dict['CL_VESTA_Enrollment_202203081316'],'CTL_ENROLL_RAW_TEST')
# except Exception as e:
#         print(e)  




CL_VESTA_Claims_Monthly_202203081839
CL_VESTA_Enrollment_202203081316
CL_Vesta_Pharmacy_Catchup_202203081328
{'CL_VESTA_Claims_Monthly_202203081839': [], 'CL_VESTA_Enrollment_202203081316': [], 'CL_Vesta_Pharmacy_Catchup_202203081328': []}


In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
df_dict['CL_VESTA_Claims_Monthly_202203081839'][df_dict['CL_VESTA_Claims_Monthly_202203081839']['PROCEDURE_CODE'].str.contains('0013A')]

,CLAIM NUMBER,CLAIM LINE NUMBER,LINE_OF_BUSINESS,UNIQUE PATIENT IDENTIFIER,PATIENT DATE OF BIRTH,PATIENT SEX,TYPE OF CLAIM,POS,SERVICEDESCRIPTION,BILL TYPE,FIRST DATE OF SERVICE,LAST DATE OF SERVICE,PAID DATE,REVENUE CODE,PROCEDURE_CODE,PROCEDURE_CODE_DESCRIPTION,MODIFIER1,MODIFIER2,UNITS,ADMITTING DIAGNOSIS,PRINCIPAL DIAGNOSIS,DX1,DIAGDESC1,DX2,DIAGDESC2,DX3,DIAGDESC3,DX4,DIAGDESC4,DX5,DIAGDESC5,DX6,DIAGDESC6,DX7,DIAGDESC7,DX8,DIAGDESC8,DX9,DIAGDESC9,PRIMARY PROCEDURE,SECONDARY PROCEDURE,DISCHARGE STATUS,PAYMENT DATE,PAYMENT STATUS,PROVIDER NUMBER,PARTICIPATING STATUS,TOTAL CHARGES,BILLED AMOUNT,ALLOWED AMOUNT,TOTAL AMOUNT PAID,PAYMENT INDICATOR,AUTHORIZATION NUMBER,CLAIM INDICATOR
23176,2636446,1,PACE,1017651,1945-10-15,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-07,2021-12-07,2022-02-01,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,Z23,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-01,Paid,1104087451,,50.00,50.00,40.00,40.00,33,,1500
23202,2636457,1,PACE,1017537,1960-10-15,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-07,2021-12-07,2022-02-01,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,I10,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-01,Paid,1104087451,,50.00,50.00,40.00,40.00,33,,1500
23357,2636555,1,PACE,1016409,1955-01-01,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-07,2021-12-07,2022-02-01,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,Z23,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-01,Paid,1104087451,,50.00,50.00,40.00,40.00,33,,1500
23878,2636897,2,PACE,1017538,1955-05-07,Male,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-07,2021-12-07,2022-02-01,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,E11.9,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-01,Paid,1104087451,,50.00,50.00,40.00,40.00,33,,1500
27223,2638765,2,PACE,1016159,1944-03-31,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-08,2021-12-08,2022-02-01,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,E11.9,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-01,Paid,1104087451,,50.00,50.00,40.00,40.00,33,,1500
36405,2646866,1,PACE,64630,1941-12-13,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-02,2021-12-02,2022-02-08,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,Z23,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-08,Paid,1083727762,,40.00,40.00,.00,40.00,33,,1500
36862,2647152,2,PACE,1017332,1957-09-01,Male,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-01,2021-12-01,2022-02-08,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,M54.50,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-08,Paid,1982078499,,40.00,40.00,40.00,40.00,33,,1500
37611,2647606,3,PACE,1017965,1957-01-08,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-14,2021-12-14,2022-02-08,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,Z23,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-08,Paid,1104087451,,.01,.01,.01,.01,33,,1500
45032,2651532,2,PACE,1016718,1963-03-04,Female,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-16,2021-12-16,2022-02-15,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,Z23,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-15,Paid,1104087451,,.01,.01,.01,.01,33,,1500
45247,2651642,3,PACE,1017891,1959-06-16,Male,2,11,NOT FOR ORDERS Coronavirus Infusion Service/Va...,,2021-12-16,2021-12-16,2022-02-15,,0013A,Moderna Covid-19 Vaccine Administration – Thir...,,,1.00,,E78.2,Z23,Encounter for immunization,,,,,,,,,,,,,,,,,,,,2022-02-15,Paid,1518198944,,50.00,50.00,40.00,40.00,33,,1500


In [5]:
claimslog

['Successfully created CTL File Temporary Folder at 2022-04-05 13:50:02',
 'AWS connection object created at 2022-04-05 13:50:03',
 'Files were successfully downloaded at 2022-04-05 13:50:10',
 'Successfully Deleted all contents in temporary CTL Folder at 2022-04-05 13:50:13']

In [ ]:
my_string = '\n'.join(claimslog)

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "VestaPingLog@gmail.com"  # Enter your address
receiver_email_list = ["jdriggers@vestahealthcare.com","john@vestahealthcare.com",'joe@vestahealthcare.com']  # Enter receiver address
password = os.getenv('Vesta_Ping_Log_Email')
message = "Subject: Ping Logs \n" + '''
             
''' + my_string

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    for receiver_email in receiver_email_list:
        server.sendmail(sender_email, receiver_email, message)